In [1]:
!pip install numpy matplotlib

!pip install ipython

Defaulting to user installation because normal site-packages is not writeable
Looking in links: /usr/share/pip-wheels
Defaulting to user installation because normal site-packages is not writeable
Looking in links: /usr/share/pip-wheels


# 3-Qubit Phase-Flip Code (Simulation)

We protect against phase flips \(Z\) by encoding in the X-basis:

\[
|0_L\rangle = |+++\rangle,\qquad |1_L\rangle = |---\rangle
\]

Equivalently, we can implement the phase-flip repetition code by applying Hadamard gates before/after the bit-flip repetition code.

---

### Simulation Steps

✅ 1) Encode \(|\psi\rangle\) into the phase-flip code  
✅ 2) Apply a phase-flip error \(Z_i\) on one physical qubit  
✅ 3) Measure the stabilizers \(X_1X_2\) and \(X_2X_3\) (syndrome extraction)  
✅ 4) Apply the correction \(Z_i\) and compute fidelity


In [2]:
import numpy as np

I = np.eye(2, dtype=complex)
X = np.array([[0, 1],
              [1, 0]], dtype=complex)
Z = np.array([[1, 0],
              [0, -1]], dtype=complex)

def kron(*ops):
    out = np.array([[1]], dtype=complex)
    for op in ops:
        out = np.kron(out, op)
    return out

def fidelity(psi_ref, psi_test):
    return float(np.abs(np.vdot(psi_ref, psi_test))**2)

# ================================
# Operators on qubit i in 3-qubit system
# ordering: |q2 q1 q0>
# ================================
def Z_on_qubit(i):
    if i == 0:
        return kron(I, I, Z)
    if i == 1:
        return kron(I, Z, I)
    if i == 2:
        return kron(Z, I, I)

# ================================
# Phase-flip code encoding:
# |psi> = alpha|0> + beta|1>
# -> alpha|+++> + beta|--->
# ================================
def encode_phaseflip3(alpha, beta):
    plus  = (1/np.sqrt(2)) * np.array([1, 1], dtype=complex)
    minus = (1/np.sqrt(2)) * np.array([1,-1], dtype=complex)

    ket_ppp = np.kron(np.kron(plus, plus), plus)
    ket_mmm = np.kron(np.kron(minus, minus), minus)

    return alpha * ket_ppp + beta * ket_mmm

# ================================
# Phase-flip stabilizers (X-type):
# X1X2 and X2X3
# ================================
X1X2 = kron(I, X, X)   # X(q1)X(q0)
X2X3 = kron(X, X, I)   # X(q2)X(q1)

def syndrome_phaseflip(state):
    s1 = np.real(np.vdot(state, X1X2 @ state))
    s2 = np.real(np.vdot(state, X2X3 @ state))
    s1 = 1 if s1 >= 0 else -1
    s2 = 1 if s2 >= 0 else -1
    return (s1, s2)

# Syndrome table (same pattern):
# (+1,+1) -> no error
# (-1,+1) -> Z on q0
# (-1,-1) -> Z on q1
# (+1,-1) -> Z on q2
def correction_from_syndrome_phaseflip(s):
    if s == (1, 1):
        return None
    if s == (-1, 1):
        return 0
    if s == (-1, -1):
        return 1
    if s == (1, -1):
        return 2

def correct_phaseflip(state):
    syn = syndrome_phaseflip(state)
    q = correction_from_syndrome_phaseflip(syn)
    if q is None:
        return state, syn, None
    return Z_on_qubit(q) @ state, syn, q

# ================================
# ✅ Example Run
# ================================
alpha = 0.6
beta = np.sqrt(1 - alpha**2)

psi_L = encode_phaseflip3(alpha, beta)

print("Syndrome (no error):", syndrome_phaseflip(psi_L))

for err in [None, 0, 1, 2]:
    noisy = psi_L.copy()
    if err is not None:
        noisy = Z_on_qubit(err) @ noisy

    corrected, syn, qcorr = correct_phaseflip(noisy)
    F = fidelity(psi_L, corrected)

    print("\n----------------------------")
    print("Applied Z error on qubit:", err)
    print("Measured syndrome (X1X2, X2X3):", syn)
    print("Correction applied on qubit:", qcorr)
    print("Fidelity after correction:", F)

Syndrome (no error): (1, 1)

----------------------------
Applied Z error on qubit: None
Measured syndrome (X1X2, X2X3): (1, 1)
Correction applied on qubit: None
Fidelity after correction: 0.9999999999999993

----------------------------
Applied Z error on qubit: 0
Measured syndrome (X1X2, X2X3): (-1, 1)
Correction applied on qubit: 0
Fidelity after correction: 0.9999999999999993

----------------------------
Applied Z error on qubit: 1
Measured syndrome (X1X2, X2X3): (-1, -1)
Correction applied on qubit: 1
Fidelity after correction: 0.9999999999999993

----------------------------
Applied Z error on qubit: 2
Measured syndrome (X1X2, X2X3): (1, -1)
Correction applied on qubit: 2
Fidelity after correction: 0.9999999999999993
